In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation,Input
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam, SGD
from keras.models import Model
from keras.layers.advanced_activations import PReLU
from keras.layers.convolutional import Conv2D, MaxPooling2D

from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')


# fix random seed for reproducibility
seed = 1070
np.random.seed(seed)

In [ ]:
file="../input/train.csv"
df = pd.read_csv(file)
data = df.values
df = None
np.random.shuffle(data)

InputPixels = data[:,1:].reshape(-1,28,28).astype('float32')
Desired = np_utils.to_categorical(data[:,0], 10)

num_classes = Desired.shape[1]

In [ ]:
def normalization(x, mu, sigma):
    
    x_norm = np.zeros_like(x)

    for n in range(len(x)):
        for j in range(len(x[n])):
            for k in range(len(x[n, j])):
                if(sigma[j, k]!=0):
                    x_norm[n, j, k] = (x[n, j, k] - mu[j, k]) / sigma[j, k]
                else:
                    x_norm[n, j, k] = 0
                    
    return x_norm

In [ ]:
mu = np.mean(InputPixels, axis=0)
sigma = np.max(InputPixels, axis=0)-np.min(InputPixels, axis=0)

In [ ]:
InputPixels = normalization(InputPixels, mu, sigma)

In [ ]:
InputPixels = InputPixels.reshape(-1, 1, 28, 28)

In [ ]:
porcent_valid = 0.1
VALID_SIZE = round(InputPixels.shape[0]*porcent_valid)

index_data = np.arange(InputPixels.shape[0])
np.random.shuffle(index_data)

x_train = InputPixels[index_data[VALID_SIZE:]]
x_valid = InputPixels[index_data[:VALID_SIZE]]


d_train = Desired[index_data[VALID_SIZE:]]
d_valid = Desired[index_data[:VALID_SIZE]]

InputPixels = None
Desired = None

x_train.shape

In [ ]:
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter

def elastic_transform(image):
    image = image.reshape(28,28)
    alpha = 15
    sigma = 3
    random_state=None
    """Elastic deformation of images as described in [Simard2003]_.
    .. [Simard2003] Simard, Steinkraus and Platt, "Best Practices for
       Convolutional Neural Networks applied to Visual Document Analysis", in
       Proc. of the International Conference on Document Analysis and
       Recognition, 2003.
    """
    if random_state is None:
        random_state = np.random.RandomState(None)

    shape = image.shape
    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha

    x, y = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]))
    indices = np.reshape(y+dy, (-1, 1)), np.reshape(x+dx, (-1, 1))

    return map_coordinates(image, indices, order=1).reshape(shape)

In [ ]:
epochs = 30
lrate = 0.0005
decay = 0.01

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(1, 28, 28), padding='same', activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(Conv2D(32, (3, 3), input_shape=(1, 28, 28), padding='same', activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(MaxPooling2D((2, 2), padding='valid'))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(MaxPooling2D((2, 2), padding='valid'))
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(MaxPooling2D((2, 2), padding='valid'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(units=512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=10, activation='softmax', kernel_regularizer=l2(1e-7)))

adam = Adam(lr=lrate, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
# Compile model

model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
print(model.summary())

In [ ]:
gen_train= ImageDataGenerator(preprocessing_function=elastic_transform)


In [ ]:
checkpointer = ModelCheckpoint(filepath='/tmp/weights.hdf5', verbose=1, save_best_only=True, monitor='val_acc')
rate_reduction = ReduceLROnPlateau(monitor='val_acc',patience=3, verbose=1,factor=0.5,min_lr=1e-7)

In [ ]:
batch_size=100

model_info = model.fit_generator(gen_train.flow(x_train, d_train, batch_size=batch_size),
                                 steps_per_epoch=x_train.shape[0]/batch_size, 
                                 epochs=epochs, 
                                 validation_data=(x_valid, d_valid),
                                 callbacks=[checkpointer, rate_reduction])

# Final evaluation of the model
model.load_weights("/tmp/weights.hdf5")
scores = model.evaluate(x_valid, d_valid, verbose=0)
print("Error: %.2f%%" % (1 - scores[1]))

In [ ]:
def plot_model_history(model_history):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    # summarize history for accuracy
    axs[0].plot(range(1,len(model_history.history['acc'])+1),model_history.history['acc'])
    axs[0].plot(range(1,len(model_history.history['val_acc'])+1),model_history.history['val_acc'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['acc'])+1),len(model_history.history['acc'])/10)
    axs[0].legend(['train', 'val'], loc='best')
    # summarize history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.show()
    
plot_model_history(model_info)

In [ ]:
file="../input/test.csv"
df = pd.read_csv(file)
data = df.values
df = None
x_test = data.reshape(-1,28,28).astype('float32')

In [ ]:
x_test = normalization(x_test, mu, sigma)

In [ ]:
x_test = x_test.reshape(-1, 1, 28, 28)

In [ ]:
d_test = model.predict(x_test, batch_size=32, verbose=0)

In [ ]:
import matplotlib.pyplot as plt

x = 10

plt.imshow(np.reshape(x_test[x],(28,28)))
plt.show()

d = np.argmax(d_test,axis=1)
print(d[x])

In [ ]:
#ImageId,Label
ImageId = np.arange(x_test.shape[0])+1

raw_data = {'ImageId': ImageId,
        'Label': d}
df = pd.DataFrame(raw_data, columns = ['ImageId', 'Label'])

df.to_csv(path_or_buf = 'output.csv', index=None, header=True)